import numpy as np
from sympy.physics.wigner import wigner_3j,wigner_6j
import sympy as sy
from numpy import linalg as LA
from IPython.display import Latex
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_palette('terrain')
#from YbOH_Energy_Levels_symbolic import YbOHLevels
np.set_printoptions(precision=8, suppress=True)

In [10]:
def H_hfs(q_numbers,matrix_elements,SO=0,E=0,B=0,M_values='all',precision=5):
    q_str = list(q_numbers)
    B,a,d,pq,eQq = sy.symbols('B a d pq eQq')
    size = len(q_numbers[q_str[0]])
    H = np.zeros((size,size)).tolist()
    for i in range(size):
        for j in range(size):
            state_out = {q+'0':q_numbers[q][i] for q in q_str}
            state_in = {q+'1':q_numbers[q][j] for q in q_str}
            q_args = {**state_out,**state_in}
            elements = {term: element(**q_args) for term, element in matrix_elements.items()}
            H[i][j] = B*elements['N^2'] + a*elements['IzLz_Yb'] - d*elements['T2_2(IS)_Yb']+\
                          eQq*elements['T2_0(II)_Yb']+pq*elements['Lambda-Doubling']
            # if M_values!='none':
            #     H[i][j]+=params['g_L']*params['mu_B']*Bz*elements['ZeemanLZ']+params['g_S']*params['mu_B']*Bz*elements['ZeemanSZ'] +\
            #     Bz*params['g_lp']*params['mu_B']*elements['ZeemanParityZ'] - params['muE_A']*Ez*elements['StarkZ']
            # H[i][j] = round(H[i][j],precision)


                # params['bFH']*elements['I.S'] + params['cH']*np.sqrt(6)/3*elements['T2_0(IS)']
    H_symbolic = sy.Matrix(H)
    H_func = sy.lambdify((Ez,Bz), H_symbolic, modules='numpy')
    return H_func,H_symbolic
                          
def q_numbers_aBJ(N_range,Lambda=1,S=1/2,I_list=[0,1/2],Omega_values=[1/2]):
    IYb=I_list[0]
    iH = I_list[-1]
    Nmin,Nmax=N_range[0],N_range[-1]
    Jmin = abs(Nmin-S)
    Jmax = abs(Nmax+S)
    if Nmin<abs(Lambda):
        print('Nmin must be >= L')
        Nmin=abs(Lambda)
    q_str = ['L','Sigma','Omega','J','F']
    q_numbers = {}
    for q in q_str:
        q_numbers[q] = []
    I = max(IYb,iH)
    for J in np.arange(Jmin,Jmax+1,1):
        for F in np.arange(abs(J-I),abs(J+I)+1,1):
            for Sigma in np.arange(-abs(S),abs(S)+1,1):
                for L in {True:[0], False:[-Lambda,Lambda]}[Lambda==0]:
                    Omega=L+Sigma
                    if abs(Omega) not in Omega_values:
                        continue
                    else:
                        values = [L,Sigma,Omega,J,F]
                    for q,val in zip(q_str,values):
                        q_numbers[q].append(val+0)    #looks weird but adding 0 converts -0 to 0
    return q_numbers
                          
def kronecker(a,b):         # Kronecker delta function
    if a==b:
        return 1
    else:
        return 0
    
def IL(L0,Sigma0,Omega0,J0,F0,L1,Sigma1,Omega1,J1,F1,S,I):
    if not kronecker(L0,L1)*kronecker(F0,F1)*kronecker(Sigma0,Sigma1)*kronecker(Omega0,Omega1):
        return 0
    else:
        return L0*(-1)**(J1+I+F0)*wigner_6j(J1,I,F0,I,J0,1)*\
            (-1)**(J0-Omega0)*wigner_3j(J0,1,J1,-Omega0,0,Omega1)*\
            sy.sqrt((2*J0+1)*(2*J1+1)*(2*I+1)*(I+1)*I)
    
def T2q2_IS(L0,Sigma0,Omega0,J0,F0,L1,Sigma1,Omega1,J1,F1,S,I):
    if not kronecker(F0,F1)*(not kronecker(L0,L1)):
        return 0
    else:
        return (-1)**(J1+I+F0+J0-Omega0+S-Sigma0)*\
            wigner_6j(J1,I,F0,I,J0,1)*sy.sqrt((2*J0+1)*(2*J1+1)*S*(S+1)*(2*S+1)*I*(I+1)*(2*I+1))*\
            sum([(-1)**(q)*kronecker(L0,L1-2*q)*wigner_3j(S,1,S,-Sigma0,q,Sigma1)*wigner_3j(J0,1,J1,-Omega0,-q,Omega1) for q in [-1,1]])
    
def a_pos_term(L0,Sigma0,Omega0,J0,F0,L1,Sigma1,Omega1,J1,F1,S,I):
    a_term= 1/2*(IL(L0,Sigma0,Omega0,J0,F0,L1,Sigma1,Omega1,J1,F1,S,I)+(-1)**(J0-S+J1-S)*IL(-L0,-Sigma0,-Omega0,J0,F0,-L1,-Sigma1,-Omega1,J1,F1,S,I))
    return sy.nsimplify(a_term)

def d_pos_term(L0,Sigma0,Omega0,J0,F0,L1,Sigma1,Omega1,J1,F1,S,I):
    d_term = 1/2*((-1)**(J0-S)*T2q2_IS(-L0,-Sigma0,-Omega0,J0,F0,L1,Sigma1,Omega1,J1,F1,S,I)+(-1)**(J1-S)*T2q2_IS(L0,Sigma0,Omega0,J0,F0,-L1,-Sigma1,-Omega1,J1,F1,S,I))
    return sy.nsimplify(d_term)

In [64]:
B,a,d,pq,eQq= sy.symbols('B a d (p+2q) eQq')

In [79]:
Ja = sy.nsimplify(1/2)
Jb = sy.nsimplify(3/2)
F = sy.nsimplify(1)
I = sy.nsimplify(1/2)
S = sy.nsimplify(1/2)
L = sy.nsimplify(1)

In [80]:
diag_a = sy.nsimplify(B*Ja*(Ja+1) + -(-1)**(Ja-S)*pq/2*(Ja+1/2))
diag_b = sy.nsimplify(B*Jb*(Jb+1) + -(-1)**(Jb-S)*pq/2*(Jb+1/2))

In [81]:
off_r = a*a_pos_term(1,-1/2,1/2,Jb,F,1,-1/2,1/2,Ja,F,S,I) + d*d_pos_term(1,-1/2,1/2,Jb,F,1,-1/2,1/2,Ja,F,S,I) 
off_l = a*a_pos_term(1,-1/2,1/2,Ja,F,1,-1/2,1/2,Jb,F,S,I) + d*d_pos_term(1,-1/2,1/2,Ja,F,1,-1/2,1/2,Jb,F,S,I) 

In [82]:
H=sy.Matrix([[diag_a,off_r],[off_l,diag_b]])

In [83]:
H

Matrix([
[         -(p+2q)/2 + 3*B/4, -sqrt(2)*a/3 + sqrt(2)*d/6],
[-sqrt(2)*a/3 + sqrt(2)*d/6,            (p+2q) + 15*B/4]])

In [73]:
H - sy.Matrix([[1,0],[0,1]])*(H[0,0]+H[1,1])/2

Matrix([
[         -3*(p+2q)/4 - 3*B/2, -sqrt(14)*a/3 + sqrt(14)*d/6],
[-sqrt(14)*a/3 + sqrt(14)*d/6,           3*(p+2q)/4 + 3*B/2]])

In [42]:
out=H.eigenvects(simplify=True)

In [43]:
out[0][0]

(p+2q)/4 + 9*B/4 - sqrt(81*(p+2q)**2 + 324*(p+2q)*B + 324*B**2 + 32*a**2 - 32*a*d + 8*d**2)/12

In [44]:
energies = [out[0][0],out[1][0]]

In [45]:
evects = [out[0][2][0], out[1][2][0]]

In [49]:
sy.simplify(evects[0].normalized())

Matrix([
[2*sqrt(2)*(-2*a + d)/(sqrt(8*Abs((2*a - d)/(9*(p+2q) + 18*B - sqrt(81*(p+2q)**2 + 324*(p+2q)*B + 324*B**2 + 32*a**2 - 32*a*d + 8*d**2)))**2 + 1)*(9*(p+2q) + 18*B - sqrt(81*(p+2q)**2 + 324*(p+2q)*B + 324*B**2 + 32*a**2 - 32*a*d + 8*d**2)))],
[                                                                                                                  1/sqrt(8*Abs((2*a - d)/(9*(p+2q) + 18*B - sqrt(81*(p+2q)**2 + 324*(p+2q)*B + 324*B**2 + 32*a**2 - 32*a*d + 8*d**2)))**2 + 1)]])

In [56]:
sy.simplify(evects[0].norm())

sqrt(8*Abs((2*a - d)/(9*(p+2q) + 18*B - sqrt(81*(p+2q)**2 + 324*(p+2q)*B + 324*B**2 + 32*a**2 - 32*a*d + 8*d**2)))**2 + 1)

In [63]:
y = 81*pq**2 + 324*pq*B + 324*B**2 + 32*a**2 - 32*a*d+8*d**2
sy.simplify(y)

81*(p+2q)**2 + 324*(p+2q)*B + 324*B**2 + 32*a**2 - 32*a*d + 8*d**2

In [38]:
norm_evects = []
for evec in evects:
    norm = evec.T*evec
    norm_evects.append(evec/norm)

TypeError: unsupported operand type(s) for /: 'One' and 'MutableDenseMatrix'

In [41]:
evects[0].normalize()

AttributeError: 'MutableDenseMatrix' object has no attribute 'normalize'

In [116]:
a,b,c,d= sy.symbols('a b c d')

In [123]:
G=sy.Matrix([[b,-a+d],[-a+d,c]])
G

Matrix([
[     b, -a + d],
[-a + d,      c]])

In [122]:
G.eigenvects()[0][2][0]

Matrix([
[(2*a - 2*d)/(b - c + sqrt(4*a**2 - 8*a*d + b**2 - 2*b*c + c**2 + 4*d**2))],
[                                                                        1]])

In [128]:
n = G.eigenvects()[1][2][0][0]

In [129]:
n

(2*a - 2*d)/(b - c - sqrt(4*a**2 - 8*a*d + b**2 - 2*b*c + c**2 + 4*d**2))

In [133]:
sy.simplify(sy.sqrt(n**2+1))

sqrt(4*(a - d)**2/(-b + c + sqrt(4*a**2 - 8*a*d + b**2 - 2*b*c + c**2 + 4*d**2))**2 + 1)